<center> 

<h1 style="text-align:center"> Applicative Parsing </h1>
<h2 style="text-align:center"> CSCI7000-11 S23: Principles of Functional Programming </h2>
</center>

```haskell
spaces :: Parser ()
spaces = many (satisfy isSpace) *> pure ()

char :: Char -> Parser Char
char c = spaces *> satisfy (==c)

plus, times :: Parser Char
plus = char '+'
times = char '*'

num :: Parser Int
num = spaces *> int

data Arith =
   Num Int
 | Plus Arith Arith
 | Times Arith Arith 
 deriving Show

term, factor, atom :: Parser Arith
term   =     Plus <$> factor <* plus <*> term
         <|> factor
factor =     Times <$> atom <* times <*> factor 
         <|> atom
atom   =     Num <$> num 
         <|> (char '(' *> term <* char ')')
```

```
Term   ::= Factor + Term | Factor
Factor ::= Atom * Factor | Atom
Atom   ::= n | ( Term )
```

In [1]:
module type FUNCTOR = sig
  type 'a t
  val fmap: ('a -> 'b) -> 'a t -> 'b t
end

module type FUNCTOR = sig type 'a t val fmap : ('a -> 'b) -> 'a t -> 'b t end


In [2]:
module type APPLICATIVE = sig
  include FUNCTOR
  val pure: 'a -> 'a t
  val fseq: ('a -> 'b) t -> 'a t -> 'b t
end

module type APPLICATIVE =
  sig
    type 'a t
    val fmap : ('a -> 'b) -> 'a t -> 'b t
    val pure : 'a -> 'a t
    val fseq : ('a -> 'b) t -> 'a t -> 'b t
  end


In [3]:
type 'a parser = Parser of (string -> ('a*string) option)

type 'a parser = Parser of (string -> ('a * string) option)


In [4]:
let parse (Parser f) s = f s;;
let mk_parser f = Parser f;;
let split (s:string) : (char*string) = 
  (s.[0], String.sub s 1 ((String.length s) - 1))

val parse : 'a parser -> string -> ('a * string) option = <fun>


val mk_parser : (string -> ('a * string) option) -> 'a parser = <fun>


val split : string -> char * string = <fun>


In [5]:
split "Hello"

- : char * string = ('H', "ello")


In [6]:
let letter (c:char) : char parser = mk_parser @@
  (fun (s:string) -> 
    if s="" then None
    else match split s with
      | (c',s') when c'=c -> Some (c',s')
      | _ -> None)

val letter : char -> char parser = <fun>


In [7]:
parse (letter 'c') "chocolate" (* ==> Some ('c', "hocolate") *)

- : (char * string) option = Some ('c', "hocolate")


In [8]:
parse (letter 'c') "vanilla" (* ==> Some ('c', "hocolate") *)

- : (char * string) option = None


In [9]:
String.make 1 'c' |> String.length

- : int = 1


In [10]:
let fmap (f : 'a -> 'b) (p: 'a parser) : 'b parser = mk_parser @@
  fun (s:string) -> match parse p s with
    | Some (a,s') -> Some (f a, s')
    | None -> None

val fmap : ('a -> 'b) -> 'a parser -> 'b parser = <fun>


In [11]:
let pure (x:'a) : 'a parser = mk_parser @@
  fun s -> Some (x,s)

val pure : 'a -> 'a parser = <fun>


In [12]:
let fseq (p1: ('a -> 'b) parser) (ap: 'a parser) : 'b parser = mk_parser @@
  fun (s:string) -> match parse p1 s with
    | Some (g, s') -> 
      let bp = fmap g ap in
      parse bp s'
    | None -> None

val fseq : ('a -> 'b) parser -> 'a parser -> 'b parser = <fun>


In [13]:
let (<$>) = fmap;;
let (<*>) = fseq

val ( <$> ) : ('a -> 'b) -> 'a parser -> 'b parser = <fun>


val ( <*> ) : ('a -> 'b) parser -> 'a parser -> 'b parser = <fun>


In [27]:
let chp = 
let (cp: char parser) = letter 'c' in
let (hp: char parser) = letter 'h' in
let (f: (char -> char list) parser) = (fun c h -> [c;h]) <$> cp in
let (chp: char list parser) = f <*> hp in
chp

val chp : char list parser = Parser <fun>


In [28]:
parse chp "chocolate"

- : (char list * string) option = Some (['c'; 'h'], "ocolate")


In [38]:
let mk_char_list c h = [c;h];;
parse (mk_char_list <$> letter 'c' <*> letter 'h') "chocolate"

val mk_char_list : 'a -> 'a -> 'a list = <fun>


- : (char list * string) option = Some (['c'; 'h'], "ocolate")


In [31]:
let char_to_string = (fun c -> String.make 1 c)

val char_to_string : char -> string = <fun>


In [33]:
match parse (letter 'c') "chocolate" with
| Some (c,s) -> Some (String.make 1 c, s)
| None -> None

- : (string * string) option = Some ("c", "hocolate")


In [35]:
parse (char_to_string <$> letter 'c') "chocolate"

- : (string * string) option = Some ("c", "hocolate")


In [39]:
parse ((mk_char_list <$> letter 'c') <*> letter 'h') "chocolate"

- : (char list * string) option = Some (['c'; 'h'], "ocolate")


In [14]:
(*
 * We want to write a string parser.
 *)
let str_cons (c:char) (s:string) : string = (String.make 1 c)^s

val str_cons : char -> string -> string = <fun>


In [15]:
let char = letter

val char : char -> char parser = <fun>


In [19]:
(*
 * val ( <$> ) : ('a -> 'b) -> 'a parser -> 'b parser 
 * val ( <*> ) : ('a -> 'b) parser -> 'a parser -> 'b parser
 * (str_cons) : char -> string -> string
 * (char c) : char parser
 * (string s') : string parser
 *)
 
let rec string (s:string): string parser = 
  if s="" then pure "" else
  match split s with
  | (c,s') -> (str_cons <$> (char c)) <*> (string s')

val string : string -> string parser = <fun>


In [17]:
parse (string "choc") "chocolate"

- : (string * string) option = Some ("choc", "olate")


In [18]:
parse (string "choc") "vanilla"

- : (string * string) option = None


In [20]:
let is_digit (c:char) : bool = match c with
  | '0' .. '9' -> true
  | _ -> false

val is_digit : char -> bool = <fun>


In [21]:
let satisfy (f: char -> bool) : char parser = mk_parser @@
  (fun (s:string) -> 
    if s="" then None
    else match split s with
      | (c',s') when f c' -> Some (c',s')
      | _ -> None)

val satisfy : (char -> bool) -> char parser = <fun>


In [22]:
let letter_parser_v2 (c:char) = satisfy (fun c' -> c'=c) 

val letter_parser_v2 : char -> char parser = <fun>


In [23]:
parse (letter_parser_v2 'c') "chocolate"

- : (char * string) option = Some ('c', "hocolate")


In [24]:
let digit_parser : int parser = (fun c -> int_of_string @@ String.make 1 c)<$> satisfy is_digit

val digit_parser : int parser = Parser <fun>


In [25]:
parse digit_parser "1c2"

- : (int * string) option = Some (1, "c2")


error: compile_error

In [30]:
let (<|>) (f:'a parser) (g: 'a parser) : 'a parser = mk_parser @@
  fun s -> match parse f s with
  | None -> parse g s
  | Some (a,s') -> Some (a,s')

val ( <|> ) : 'a parser -> 'a parser -> 'a parser = <fun>


In [33]:
 (*
  * digit ::= 0...9
  * more_digits ::= [] | number
  * number ::= digit.more_digits
  *)
let rec some (p: 'a parser) : 'a list parser = (fun x xs -> x::xs) <$> p <*> (mk_parser @@ fun s -> parse (many p) s)
    and many (p: 'a parser) : 'a list parser = (mk_parser @@ fun s -> parse (some p) s) <|> pure []

val some : 'a parser -> 'a list parser = <fun>
val many : 'a parser -> 'a list parser = <fun>


In [34]:
let rec char_list_to_string (cl:char list) : string = 
  match cl with
  | [] -> ""
  | c::cl' -> str_cons c (char_list_to_string cl');;
let int = (fun cl -> int_of_string @@ char_list_to_string cl) <$> some (satisfy is_digit)

val char_list_to_string : char list -> string = <fun>


val int : int parser = Parser <fun>


In [35]:
parse int "12345color"

- : (int * string) option = Some (12345, "color")


In [38]:
let int_com = (fun (x: int) (c:char) -> x) <$> int <*> char ','

val int_com : int parser = Parser <fun>


In [40]:
parse ((fun i j k -> [i; j; k]) <$> int_com <*> int_com <*> int) "1,2,3" (* ==> [1;2;3]*)

- : (int list * string) option = Some ([1; 2; 3], "")


In [41]:
let lift (ap: 'a parser)  : ('b -> 'a) parser = mk_parser @@
  fun s -> match parse ap s with 
    | Some (a,s') -> Some ((fun b -> a), s')
    | None -> None;;
let (<*) (ap: 'a parser) (bp: 'b parser) : 'a parser = 
  (lift ap) <*> bp

val lift : 'a parser -> ('b -> 'a) parser = <fun>


val ( <* ) : 'a parser -> 'b parser -> 'a parser = <fun>


In [42]:
let int_com = int <* char ',';;
parse ((fun i j k -> [i; j; k]) <$> int_com <*> int_com <*> int) "1,2,3" (* ==> [1;2;3]*)

val int_com : int parser = Parser <fun>


- : (int list * string) option = Some ([1; 2; 3], "")
